In [1]:
import numpy as np 
import matplotlib 
import matplotlib.pyplot as plt
import random
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
data = pd.read_json("datasetTest100k.json", typ="frame", lines = True, precise_float=True,nrows = 5*100000)

In [3]:
#from RealisticDataExplorer.ipynb
trkData = np.stack(data[3],axis=0)

trk_high_level_name = ["track_2_d0_significance", "track_3_d0_significance",
                       "track_2_z0_significance", "track_3_z0_significance",
                       "n_tracks_over_d0_threshold", "jet_prob", "jet_width_eta", "jet_width_phi"]

for i, trkName in enumerate(trk_high_level_name):
    tmpArr = np.array(trkData[:,i],dtype="float32")
    data[trkName] = tmpArr

vtxData = np.stack(data[4],axis=0)
    
vtx_high_level_name = ["vertex_significance", "n_secondary_vertices", "n_secondary_vertex_tracks",
                       "delta_r_vertex", "vertex_mass", "vertex_energy_fraction"]

for i, vtxName in enumerate(vtx_high_level_name):
    tmpArr = np.array(vtxData[:,i],dtype="float32")
    data[vtxName] = tmpArr

In [4]:
jets_trk = []
for i in range(0,200000):
    trkVars = []
    for j in range(len(data[5][i])):
        trkVars.append(data[5][i][j][0])
    jets_trk.append(trkVars)

In [5]:
d0 = []
for i in jets_trk:
    temp = []
    for j in i:
        temp.append(j[0])
    d0.append(temp)

for i in d0:
    while len(i)<15:
        i.append(0)
    while len(i)>15:
        i.remove(min(i))
X_d0 = torch.tensor(d0,dtype=torch.float32)

In [6]:
def sortTrk (data):
    for i in data:
        i.sort(key = lambda ls : np.abs(ls[0]))

def padding (data, n, length):
    temp = []
    for i in range(length):
        temp.append(0)
    for i in data:
        while len(i)<n:
            i.append(temp)
        while len(i)>n:
            i.remove(i[len(i)-1])
    return 

In [7]:
sortTrk(jets_trk)
padding(jets_trk, 15, 5)

In [8]:
y = data[2]
y = y.values.tolist()

for i in range(len(y)):
    if y[i]==5:
        y[i]=1
    else:
        y[i]=0

In [9]:
X_trk = torch.tensor(jets_trk, dtype=torch.float32)
X_trk = torch.flatten(X_trk,start_dim=1)
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

In [10]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [11]:
Ndimentions=15
n_epochs=100
batch_size=100

train_ds = TensorDataset(X_d0,y)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

NNmodel = nn.Sequential(
    nn.Linear(Ndimentions,Ndimentions),
    nn.ReLU(),
    nn.Dropout(p=0.3),
    nn.Linear(Ndimentions,Ndimentions),
    nn.ReLU(),
    nn.Dropout(p=0.3),
    nn.Linear(Ndimentions,Ndimentions),
    nn.ReLU(),
    nn.Linear(Ndimentions,Ndimentions),
    nn.ReLU(),
    nn.Linear(Ndimentions,Ndimentions),
    nn.ReLU(),
    nn.Linear(Ndimentions,Ndimentions),
    nn.ReLU(),
    nn.Linear(Ndimentions,Ndimentions),
    nn.ReLU(),
    nn.Linear(Ndimentions,Ndimentions),
    nn.ReLU(),
    nn.Linear(Ndimentions,Ndimentions),
    nn.ReLU(),
    nn.Linear(Ndimentions,1),
    nn.Sigmoid()
    )

def lr_lambda(epoch):
    if epoch <= 80:
        return 1
    else:
        return 100*(-0.00045*epoch+0.046)
        
loss_fn = nn.BCELoss()
optimizer = optim.SGD(NNmodel.parameters(), lr = 0.01)
lr_scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda, verbose=True)

for epoch in range(n_epochs):
    NNmodel.train()
    for xb, yb in train_dl:
        y_pred = NNmodel(xb)
        loss = loss_fn(y_pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    lr_scheduler.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')

with torch.no_grad():
    y_pred = NNmodel(X_d0)
print(y_pred)
accuracy = (y_pred.round() == y).float().mean()
print(f"Accuracy {accuracy}")

Adjusting learning rate of group 0 to 1.0000e-02.
Adjusting learning rate of group 0 to 1.0000e-02.
Finished epoch 0, latest loss 0.6850210428237915
Adjusting learning rate of group 0 to 1.0000e-02.
Finished epoch 1, latest loss 0.6833359003067017
Adjusting learning rate of group 0 to 1.0000e-02.
Finished epoch 2, latest loss 0.7024783492088318
Adjusting learning rate of group 0 to 1.0000e-02.
Finished epoch 3, latest loss 0.6848697662353516
Adjusting learning rate of group 0 to 1.0000e-02.
Finished epoch 4, latest loss 0.7082681059837341
Adjusting learning rate of group 0 to 1.0000e-02.
Finished epoch 5, latest loss 0.686998724937439
Adjusting learning rate of group 0 to 1.0000e-02.
Finished epoch 6, latest loss 0.6759951114654541
Adjusting learning rate of group 0 to 1.0000e-02.
Finished epoch 7, latest loss 0.6866924166679382
Adjusting learning rate of group 0 to 1.0000e-02.
Finished epoch 8, latest loss 0.7135294079780579
Adjusting learning rate of group 0 to 1.0000e-02.
Finished e